# Configuration and Utilities


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from matplotlib.dates import date2num
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras

import re
import math

In [ ]:
# Date parsing
def dateparse (time_in_secs):    
    return datetime.fromtimestamp(float(time_in_secs))

In [ ]:
def highlights_attacks (df, attacks, ax):
  for index, row in attacks.iterrows():
    start = datetime.fromtimestamp(row['start'])
    end = datetime.fromtimestamp(row['end'])

    period = df[(df.index > start) & (df.index <= end)].index

    ax.axvspan(start, end, facecolor='pink', edgecolor='none', alpha=.4)

In [ ]:
def clean_header (header_name, label):
    x = re.match(r"\(\'value\',\s\'(\w+)\-\S*\-\S*\'\)", header_name)
    if x == None:
        return header_name
    return x.group(1) + '-' + label

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs).astype('float32'), np.array(ys).astype('float32')

In [ ]:
def return_rmse(test, predicted, model):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error for {} model is {}.".format(model, rmse))
    return rmse

In [ ]:
attacks = pd.DataFrame(
    {
        "name": ['First Attack'],
        "start" : [1619499815],
        "end" : [1619499840]
    }
)

# Fetching datasets

## Application Latency Dataset



### Application Latency read and clean

In [ ]:
application_latency = pd.read_csv('/kaggle/input/chaosml/application_latency_df.csv', parse_dates=['timestamp'], date_parser=dateparse)
application_latency.head()

In [ ]:
application_latency_pivoted = pd.pivot_table(data=application_latency,index=['timestamp'], columns=['handler'])
application_latency_pivoted.head()

In [ ]:
application_latency_flattened = pd.DataFrame(application_latency_pivoted.to_records())
application_latency_flattened.columns = [hdr.replace("('value', ", '').replace(')', '').replace("\'", '') for hdr in application_latency_flattened.columns]
application_latency_flattened.head()

In [ ]:
# may move after merge
application_latency_flattened['attack'] = False

for index, row in attacks.iterrows():
  start = datetime.fromtimestamp(row['start'])
  end = datetime.fromtimestamp(row['end'])
  application_latency_flattened.loc[(application_latency_flattened['timestamp'] > start) & (application_latency_flattened['timestamp'] <= end), 'attack'] = True

application_latency_flattened.head()

### Application Latency Plot

In [ ]:
indexed_application_latency_flattened = application_latency_flattened
indexed_application_latency_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_application_latency_flattened.plot()

highlights_attacks (indexed_application_latency_flattened, attacks, ax)


## Application Error Dataset

### Application Error read and clean

In [ ]:
application_error = pd.read_csv('/kaggle/input/chaosml/application_error.csv', parse_dates=['timestamp'], date_parser=dateparse)
application_error.head()

### Application Error Plot

In [ ]:
indexed_application_error = application_error
indexed_application_error.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_application_error.plot()

highlights_attacks (indexed_application_error, attacks, ax)

## Application Traffic dataset

In [ ]:
application_traffic = pd.read_csv('/kaggle/input/chaosml/application_traffic.csv', parse_dates=['timestamp'], date_parser=dateparse)
application_traffic.head()

In [ ]:
application_traffic_pivoted = pd.pivot_table(data=application_traffic,index=['timestamp'], columns=['code'])
application_traffic_pivoted.head()

In [ ]:
application_traffic_flattened = pd.DataFrame(application_traffic_pivoted.to_records())
application_traffic_flattened.columns = [hdr.replace("('value', ", 'code_').replace(')', '').replace("\'", '') for hdr in application_traffic_flattened.columns]
application_traffic_flattened.head()

### Application Traffic plot

In [ ]:
indexed_application_traffic_flattened = application_traffic_flattened
indexed_application_traffic_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_application_traffic_flattened.plot()

highlights_attacks (indexed_application_traffic_flattened, attacks, ax)


# Merge Datasets

Documentation about `merge_asof` function may be found [here](https://towardsdatascience.com/how-to-merge-not-matching-time-series-with-pandas-7993fcbce063)

In [ ]:
merged_latency_and_error = pd.merge_asof (application_latency_flattened, application_error, on='timestamp', tolerance=pd.Timedelta('1s'))
merged_latency_and_error_renamed = merged_latency_and_error.rename(columns={'value': 'error_rate'})
merged_latency_and_error_renamed.head()

In [ ]:
merged_latency_error_and_traffic = pd.merge_asof (merged_latency_and_error_renamed, application_traffic_flattened, on='timestamp', tolerance=pd.Timedelta('1s'))
merged_latency_error_and_traffic.head()

In [ ]:
merged_latency_error_and_traffic.describe().transpose()

### Replace Missing Values

In [ ]:
# replace with last value

colums_to_replace_with_last_value = ['error_rate','set-currency']

for column in colums_to_replace_with_last_value:
    merged_latency_error_and_traffic[column].replace(np.nan, merged_latency_error_and_traffic[column].mean(), inplace=True)

    
# replace with max value

columns_to_replace_with_max_val = ['code_500', 'checkout', 'get-cart', 'post-cart']

for column in columns_to_replace_with_max_val:
    merged_latency_error_and_traffic[column].replace(np.nan, merged_latency_error_and_traffic[column].max(), inplace=True)
    
# replace with min value

columns_to_replace_with_max_val = ['empty-cart']

for column in columns_to_replace_with_max_val:
    merged_latency_error_and_traffic[column].replace(np.nan, merged_latency_error_and_traffic[column].min(), inplace=True)

In [ ]:
merged_latency_error_and_traffic.isnull().sum(axis = 0)

### Merge Plot

In [ ]:
indexed_merge = merged_latency_error_and_traffic
indexed_merge.set_index('timestamp', inplace=True)

In [ ]:
indexed_merge.plot(subplots=True, y=['home','checkout','code_200'])

# Train a model

In [ ]:
f_predicted = 'code_200'

## Pre processing

We’ll use the last 10% of the data for testing:

In [ ]:
train_size = int(len(indexed_merge) * 0.9)
test_size = len(indexed_merge) - train_size

train, test = indexed_merge.iloc[0:train_size], indexed_merge.iloc[train_size:len(indexed_merge)]

print(len(train), len(test))

We’ll scale some of the features we’re using for our modeling:

In [ ]:
f_columns = indexed_merge.columns.to_list()
f_columns.remove('attack')
f_columns.remove(f_predicted)

f_transformer = RobustScaler()
code_200_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
code_200_transformer = code_200_transformer.fit(train[[f_predicted]])

train.loc[:, f_columns] = f_transformer.transform(
  train[f_columns].to_numpy()
)
train[f_predicted] = code_200_transformer.transform(train[[f_predicted]])

test.loc[:, f_columns] = f_transformer.transform(
  test[f_columns].to_numpy()
)
test[f_predicted] = code_200_transformer.transform(test[[f_predicted]])

print(len(train), len(test))
print(train.shape, test.shape)

In [ ]:
time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train[f_predicted], time_steps)
X_test, y_test = create_dataset(test, test[f_predicted], time_steps)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## Prediction with LSTM

In [ ]:
model_lstm = keras.Sequential()

# First LSTM layer with Dropout regularisation
model_lstm.add(keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(keras.layers.Dropout(0.2))
# Second LSTM layer
model_lstm.add(
  keras.layers.LSTM(
    units=128,
    input_shape=(X_train.shape[1], X_train.shape[2])
  )
)
model_lstm.add(keras.layers.Dropout(rate=0.2))
# The output layer
model_lstm.add(keras.layers.Dense(units=1))

model_lstm.compile(optimizer='rmsprop',loss='mean_squared_error')

In [ ]:
history_lstm = model_lstm.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
model_lstm.summary()

In [ ]:
y_pred_lstm = model_lstm.predict(X_test)
y_pred_lstm.shape

In [ ]:
y_train_inv_lstm = code_200_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv_lstm = code_200_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv_lstm = code_200_transformer.inverse_transform(y_pred_lstm)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv_lstm.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv_lstm.flatten(), label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv_lstm.flatten(), 'r', label="prediction")
plt.ylabel('200 Responses Rate')
plt.xlabel('Time Step')
plt.title('LSTM')
plt.legend()
plt.show();

In [ ]:
rmse_lstm = return_rmse(y_test,y_pred_inv_lstm,'LSTM')

## Prediction with Bidirectional LSTM with a Dropout layer

In [ ]:
model_lstm_bidirectional = keras.Sequential()

# First LSTM layer with Dropout regularisation
model_lstm_bidirectional.add(keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm_bidirectional.add(keras.layers.Dropout(0.2))
# Second LSTM layer
model_lstm_bidirectional.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=50,
      input_shape=(X_train.shape[1], X_train.shape[2])
    )
  )
)
model_lstm_bidirectional.add(keras.layers.Dropout(rate=0.2))
# The output layer
model_lstm_bidirectional.add(keras.layers.Dense(units=1))

model_lstm_bidirectional.compile(optimizer='rmsprop',loss='mean_squared_error')

In [ ]:
history_1 = model_lstm_bidirectional.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
model_lstm_bidirectional.summary()

In [ ]:
y_pred = model_lstm_bidirectional.predict(X_test)
y_pred.shape

In [ ]:
y_train_inv = code_200_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv = code_200_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv = code_200_transformer.inverse_transform(y_pred)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv.flatten(), label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv.flatten(), 'r', label="prediction")
plt.ylabel('200 Responses Rate')
plt.xlabel('Time Step')
plt.title('Bidirectional LSTM')
plt.legend()
plt.show();

In [ ]:
rmse_bi_lstm = return_rmse(y_test,y_pred_inv,'Bidirectional LSTM')

## Prediction with GRU

In [ ]:
model_gru = keras.Sequential()

# First GRU layer with Dropout regularisation
model_gru.add(keras.layers.GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), activation='tanh'))
model_gru.add(keras.layers.Dropout(0.2))
# Second LSTM layer
model_gru.add(
  keras.layers.GRU(
    units=128,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    activation='tanh'
  )
)
model_gru.add(keras.layers.Dropout(rate=0.2))
# The output layer
model_gru.add(keras.layers.Dense(units=1))

model_gru.compile(optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False), loss='mean_squared_error')

In [ ]:
history_gru = model_lstm_bidirectional.fit(
    X_train, y_train,
    epochs=50,
    batch_size=150,
    shuffle=False
)

In [ ]:
model_gru.summary()

In [ ]:
y_pred_gru = model_gru.predict(X_test)
y_pred_gru.shape

In [ ]:
y_train_inv_gru = code_200_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv_gru = code_200_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv_gru = code_200_transformer.inverse_transform(y_pred_gru)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv_gru.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv_gru.flatten(), label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv_gru.flatten(), 'r', label="prediction")
plt.ylabel('200 Responses Rate')
plt.xlabel('Time Step')
plt.title('GRU')
plt.legend()
plt.show();

In [ ]:
rmse_gru = return_rmse(y_test,y_pred_inv_gru,'GRU')

## Compare Performance

In [ ]:
rmse_df = pd.DataFrame({'Model':['LSTM', 'Bidirectional LSTM', 'GRU'], 'RMSE':[rmse_lstm, rmse_bi_lstm, rmse_gru]})
ax = rmse_df.plot.bar(x='Model', y='RMSE', rot=0)

ax.legend(bbox_to_anchor=(1.0, 1.0))

for p in ax.patches:
    b = p.get_bbox()
    val = "{:.2f}".format(b.y1 + b.y0)        
    ax.annotate(val, ((b.x0 + b.x1)/2 - 0.07, b.y1 + 0.005))